In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
import matplotlib.pyplot as plt

# 加载并清洗数据

In [ ]:
# 1. 数据预处理
# 加载训练数据集（假设训练数据集文件名为 iris_train.csv）
train_df = pd.read_csv('./data/iris_train.csv')

# 数据清洗：检查缺失值和异常值
# 鸢尾花数据集通常没有缺失值和异常值，这里仅作为示例
train_df_cleaned = train_df.dropna()  # 删除缺失值

# 特征选择：选择所有特征
X_train = train_df_cleaned.drop(columns=['Species'])  # 标签列名为 'species'
y_train = train_df_cleaned['Species']

# 数据标准化/归一化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)


## 其他标准化方法：MinMaxScaler, RobustScaler等
# 使用中位数和四分位间距（IQR）进行缩放，因此对于异常值具有鲁棒性。当数据中有许多异常值时，这是一个很好的选择。
from sklearn.preprocessing import RobustScaler
robust_scaler = RobustScaler()
X_robust_scaled = robust_scaler.fit_transform(X_train.copy(deep=True))

# 检查缺失值和异常值的方法

In [ ]:
# 检查每列的缺失值数量
data = train_df.copy(deep=True) # 深拷贝，防止原来的程序逻辑受到影响
missing_values = data.isnull().sum()
print(missing_values)

# 或者显示包含缺失值的列
columns_with_missing = missing_values[missing_values > 0]
print(columns_with_missing)

# 删除所有含有缺失值的行
data_cleaned = data.dropna()

# 或者用均值填充数值型列的缺失值
data['column_name'].fillna(data['column_name'].mean(), inplace=True)

## (1) 使用IQR方法检测异常值
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1
# 定义边界
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
# 找出异常值
outliers = ((data < lower_bound) | (data > upper_bound)).any(axis=1)
print(outliers[outliers].index)

## (2) 使用 Z-Score 方法检测异常值
from scipy import stats
import numpy as np
z_scores = np.abs(stats.zscore(data.select_dtypes(include=[np.number])))
outliers = (z_scores > 3).any(axis=1)
print(outliers[outliers].index)

# 删除异常值
data_no_outliers = data[~outliers]

# 或者用特定值替换异常值
value_to_replace = 0 # 你可以替换成你想要的值
data.loc[outliers, 'column_name'] = value_to_replace

# 保存清洗后的训练数据到指定文件 

In [68]:

preprocessed_train_data = pd.concat([pd.DataFrame(X_train_scaled, columns=X_train.columns), pd.Series(y_train, name='species')], axis=1)
preprocessed_train_data.to_csv('./output/cleaned_train_data.csv', index=False)


# 模型训练
> 数据分类在数据挖掘中是一种常见场景，您可以尝试使用不同建模方式(决策树模型,随机森林，SVC。。。。。。)

In [69]:
# 2. 建模及模型训练
# 选择模型：支持向量机（SVM）
# model = SVC(kernel='linear', random_state=42)
# model = RandomForestClassifier()
model = DecisionTreeClassifier()

# 模型训练
model.fit(X_train_scaled, y_train)

#print("支持向量:")
#print(model.support_vectors_)

# 打印截距
#print("截距:")
#print(model.intercept_)

# 打印支持向量的索引
#print("支持向量的索引:")
#print(model.support_)

# 打印每个类的支持向量数量
# print("每个类的支持向量数量:")
# print(model.n_support_)

DecisionTreeClassifier()

In [70]:
# 计算训练集上的准确率
train_accuracy = model.score(X_train_scaled, y_train)
print(f"训练集准确率: {train_accuracy:.2f}")

训练集准确率: 1.00


In [71]:
# 3. 模型推理
# 加载验证数据集（假设验证数据集文件名为 iris_val.csv）
val_df = pd.read_csv('./data/iris_test.csv')

# 数据清洗：检查缺失值和异常值
val_df_cleaned = val_df.dropna()  # 删除缺失值

# 特征选择：选择所有特征
X_val = val_df_cleaned.drop(columns=['Species'])  # 标签列名为 'species'

# 数据标准化/归一化
X_val_scaled = scaler.transform(X_val)

# 模型推理
y_pred = model.predict(X_val_scaled)

# 保存推理结果到指定目录(确保文件名，字段名正确)

In [72]:
# 保存推理结果
results = pd.DataFrame({'Sample_ID': range(len(y_pred)), 'Predicted_Species': y_pred})
results.to_csv('./output/test_data_predictions.csv', index=False)

# 试试检查您训练的结果
> 真实竞赛中会根据模型准确率评分，所以提供的推理数据不会包含标签列验证,您竞赛中不需要完成下面步骤

In [73]:
val_df_cleaned['pred']=y_pred
val_df_cleaned[['Species','pred']]
# 提取验证集的真实标签和预测结果
y_true = val_df_cleaned['Species']
y_pred = val_df_cleaned['pred']
# 评估模型
accuracy = accuracy_score(y_true, y_pred)
print(f"模型准确率: {accuracy:.2f}")
print("分类报告:")
print(classification_report(y_true, y_pred))

模型准确率: 1.00
分类报告:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00        11
   virginica       1.00      1.00      1.00         9

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



# <font color="red">最后您需要将这个notebook保存到output目录(可以通过左边的文件管理栏,或文件菜单中的另存为)，供评分使用<font color="red">